In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import joblib

### Cargar datos, preprocesarlos y entrenar modelo

In [85]:
# Esta sección se encarga de la carga de los datos, la aplicación de técnicas de preprocesamiento de texto esenciales
# y el entrenamiento de un modelo de clasificación de texto utilizando el algoritmo de Regresión Logística.

# Descargar recursos de NLTK si no están presentes
# NLTK (Natural Language Toolkit) es una librería fundamental para el procesamiento de lenguaje natural en Python.
# Este bloque asegura que los recursos necesarios para el preprocesamiento (stopwords y WordNet) estén descargados.
try:
    stopwords.words('english') # Intenta acceder a la lista de stopwords en inglés.
    WordNetLemmatizer().lemmatize('running') # Intenta usar la función de lematización.
except LookupError:
    print("Descargando recursos de NLTK...")
    nltk.download('stopwords') # Descarga la lista de stopwords si no se encuentra.
    nltk.download('wordnet') # Descarga el léxico WordNet si no se encuentra.
    print("Recursos de NLTK descargados.")

In [86]:
# Cargar y explorar los datos
# Se utiliza la librería pandas para cargar el dataset desde un archivo CSV.
file_path = 'data/training_data.csv' # Define la ruta al archivo de entrenamiento.
df = pd.read_csv(file_path, header=None, names=['label', 'text'], sep='\t')
# El archivo se carga sin encabezado y se asignan nombres a las columnas: 'label' (la clase a predecir) y 'text' (el contenido textual).
# La separación de los datos en el archivo es por tabulador ('\t').

print("Primeras filas del dataset:")
print(df.head()) # Muestra las primeras filas del DataFrame para inspeccionar los datos.
print("\nInformación del dataset:")
print(df.info()) # Proporciona un resumen conciso del DataFrame, incluyendo el tipo de datos y la cantidad de entradas no nulas.
print("\nDistribución de las etiquetas:")
print(df['label'].value_counts()) # Muestra la frecuencia de cada valor único en la columna 'label', lo que ayuda a entender el balance de clases.

Primeras filas del dataset:
   label                                               text
0      0  donald trump sends out embarrassing new year‚s...
1      0  drunk bragging trump staffer started russian c...
2      0  sheriff david clarke becomes an internet joke ...
3      0  trump is so obsessed he even has obama‚s name ...
4      0  pope francis just called out donald trump duri...

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34152 entries, 0 to 34151
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   34152 non-null  int64 
 1   text    34152 non-null  object
dtypes: int64(1), object(1)
memory usage: 533.8+ KB
None

Distribución de las etiquetas:
label
0    17572
1    16580
Name: count, dtype: int64


In [87]:
# Preprocesamiento del texto
# Esta etapa es crucial para limpiar y normalizar el texto antes de alimentarlo al modelo de aprendizaje automático.
stop_words = set(stopwords.words('english')) # Crea un conjunto de palabras comunes en inglés que se eliminarán. La conversión a conjunto mejora la eficiencia de la búsqueda.
lemmatizer = WordNetLemmatizer() # Inicializa el lematizador, que reduce las palabras a su forma base o lema.

def clean_text(text):
    # Esta función toma un texto como entrada y aplica una serie de pasos de limpieza.
    text = text.lower() # Convierte todo el texto a minúsculas para asegurar la uniformidad.
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Utiliza una expresión regular para eliminar cualquier carácter que no sea una letra (mayúscula o minúscula) o un espacio en blanco. Esto elimina números y signos de puntuación.
    tokens = text.split() # Divide el texto en una lista de palabras (tokens) utilizando los espacios como delimitadores.
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] # Aplica la lematización a cada palabra que no esté en la lista de stopwords.
    return " ".join(tokens) # Une los tokens limpios y lematizados de nuevo en una cadena de texto, separados por espacios.

df['cleaned_text'] = df['text'].apply(clean_text) # Aplica la función clean_text a cada elemento de la columna 'text' del DataFrame, creando una nueva columna llamada 'cleaned_text' con el texto preprocesado.
print("\nTexto preprocesado (ejemplo):")
print(df[['text', 'cleaned_text']].head()) # Muestra las primeras filas comparando el texto original con el texto preprocesado.


Texto preprocesado (ejemplo):
                                                text  \
0  donald trump sends out embarrassing new year‚s...   
1  drunk bragging trump staffer started russian c...   
2  sheriff david clarke becomes an internet joke ...   
3  trump is so obsessed he even has obama‚s name ...   
4  pope francis just called out donald trump duri...   

                                        cleaned_text  
0  donald trump sends embarrassing new year eve m...  
1  drunk bragging trump staffer started russian c...  
2  sheriff david clarke becomes internet joke thr...  
3  trump obsessed even obamas name coded website ...  
4  pope francis called donald trump christmas speech  


In [88]:
# Dividir los datos en conjuntos de entrenamiento y prueba
# Es fundamental dividir el dataset en un conjunto de entrenamiento para entrenar el modelo y un conjunto de prueba para evaluar su rendimiento en datos no vistos.
X = df['cleaned_text'] # Define la columna 'cleaned_text' como la variable predictora (características).
y = df['label'] # Define la columna 'label' como la variable objetivo (la clase a predecir).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# La función train_test_split divide los datos.
# test_size=0.2 indica que el 20% de los datos se utilizará para la prueba.
# random_state=42 asegura que la división sea la misma cada vez que se ejecuta el código, lo que es importante para la reproducibilidad.
# stratify=y asegura que la proporción de las clases en el conjunto de prueba sea la misma que en el conjunto de datos original, lo cual es crucial para datasets desbalanceados.

print(f"\nTamaño del conjunto de entrenamiento: {X_train.shape}") # Muestra el número de muestras en el conjunto de entrenamiento.
print(f"Tamaño del conjunto de prueba: {X_test.shape}") # Muestra el número de muestras en el conjunto de prueba.


Tamaño del conjunto de entrenamiento: (27321,)
Tamaño del conjunto de prueba: (6831,)


In [89]:
# Vectorización del texto usando TF-IDF
# Los modelos de aprendizaje automático no pueden trabajar directamente con texto, por lo que necesitamos convertirlo en representaciones numéricas.
# TF-IDF (Term Frequency-Inverse Document Frequency) es una técnica común para asignar un peso a cada palabra en un documento dentro de una colección de documentos (el corpus).
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Inicializa el vectorizador TF-IDF.
# max_features=5000 limita el número de características (palabras únicas) a las 5000 más frecuentes, lo que ayuda a reducir la dimensionalidad y el ruido. Este parámetro puede ajustarse.
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train) # Aprende el vocabulario del conjunto de entrenamiento y transforma los datos de entrenamiento en una matriz TF-IDF.
X_test_tfidf = tfidf_vectorizer.transform(X_test) # Utiliza el vocabulario aprendido del conjunto de entrenamiento para transformar los datos de prueba en la misma representación TF-IDF. Es importante usar el mismo vectorizador para ambos conjuntos para asegurar la consistencia de las características.

print("\nForma de la matriz TF-IDF para el conjunto de entrenamiento:", X_train_tfidf.shape) # Muestra la dimensión de la matriz TF-IDF del conjunto de entrenamiento (número de muestras x número de características).
print("Forma de la matriz TF-IDF para el conjunto de prueba:", X_test_tfidf.shape) # Muestra la dimensión de la matriz TF-IDF del conjunto de prueba.


Forma de la matriz TF-IDF para el conjunto de entrenamiento: (27321, 5000)
Forma de la matriz TF-IDF para el conjunto de prueba: (6831, 5000)


In [90]:
# Modelado: Regresión Logística
# Se elige el modelo de Regresión Logística, un algoritmo lineal que es eficaz para problemas de clasificación de texto.
logistic_regression_model = LogisticRegression(random_state=42) # Inicializa el modelo de Regresión Logística. random_state asegura la reproducibilidad del entrenamiento.
logistic_regression_model.fit(X_train_tfidf, y_train) # Entrena el modelo utilizando la representación TF-IDF del conjunto de entrenamiento y las etiquetas correspondientes.
y_pred_lr = logistic_regression_model.predict(X_test_tfidf) # Realiza predicciones sobre el conjunto de prueba transformado con TF-IDF.

In [91]:
# Evaluación del modelo
# Se evalúa el rendimiento del modelo utilizando métricas comunes para la clasificación.
accuracy_lr = accuracy_score(y_test, y_pred_lr) # Calcula la precisión, que es el porcentaje de predicciones correctas.
report_lr = classification_report(y_test, y_pred_lr) # Genera un informe de clasificación que incluye precisión, recall, F1-score y soporte para cada clase.

print("\nResultados del modelo de Regresión Logística:")
print(f"Precisión: {accuracy_lr:.4f}") # Muestra la precisión con cuatro decimales.
print("\nReporte de Clasificación:\n", report_lr) # Muestra el informe de clasificación completo.


Resultados del modelo de Regresión Logística:
Precisión: 0.9300

Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93      3515
           1       0.92      0.94      0.93      3316

    accuracy                           0.93      6831
   macro avg       0.93      0.93      0.93      6831
weighted avg       0.93      0.93      0.93      6831



### Análisis de sentimiento

In [92]:
# Esta sección implementa un análisis de sentimiento básico utilizando la librería VADER (Valence Aware Dictionary and sEntiment Reasoner),
# que está específicamente sintonizada para el sentimiento expresado en las redes sociales.
try:
    SentimentIntensityAnalyzer() # Intenta inicializar el analizador de sentimiento VADER.
except LookupError:
    print("Descargando léxico VADER...")
    nltk.download('vader_lexicon') # Descarga el léxico VADER si no se encuentra.
    print("Léxico VADER descargado.")

sentiment_analyzer = SentimentIntensityAnalyzer() # Inicializa el analizador de sentimiento.
df['sentiment_score'] = df['text'].apply(lambda text: sentiment_analyzer.polarity_scores(text)['compound'])
# Aplica el analizador de sentimiento a cada texto en la columna 'text'.
# polarity_scores devuelve un diccionario con las puntuaciones de sentimiento negativo, neutral, positivo y compuesto.
# Se extrae la puntuación 'compound', que es una métrica normalizada y ponderada del sentimiento general.
print("\nEjemplo de puntaje de sentimiento:")
print(df[['text', 'sentiment_score']].head()) # Muestra las primeras filas con el texto original y su puntuación de sentimiento.


Ejemplo de puntaje de sentimiento:
                                                text  sentiment_score
0  donald trump sends out embarrassing new year‚s...          -0.7096
1  drunk bragging trump staffer started russian c...          -0.3400
2  sheriff david clarke becomes an internet joke ...          -0.2960
3  trump is so obsessed he even has obama‚s name ...          -0.3052
4  pope francis just called out donald trump duri...           0.0000


### Testear con archivo testing_data.csv

In [93]:
# Esta sección carga un nuevo conjunto de datos de prueba y aplica el mismo preprocesamiento y el modelo entrenado para realizar predicciones en datos nuevos.
testing_file_path = 'data/testing_data.csv' # Define la ruta al archivo de prueba.
df_test = pd.read_csv(testing_file_path, header=None, names=['marker', 'text'], sep='\t')
# Carga el archivo de prueba de manera similar al archivo de entrenamiento, con columnas 'marker' y 'text'.

print("Primeras filas del testing_data:")
print(df_test.head()) # Muestra las primeras filas del DataFrame de prueba.
print("\nInformación del testing_data:")
print(df_test.info()) # Proporciona información sobre el DataFrame de prueba.
df_test['cleaned_text'] = df_test['text'].apply(clean_text) # Aplica la misma función de limpieza de texto al conjunto de datos de prueba.
print("\nTexto preprocesado del testing_data (ejemplo):")
print(df_test[['text', 'cleaned_text']].head()) # Muestra un ejemplo del texto preprocesado en el conjunto de datos de prueba.

Primeras filas del testing_data:
  marker                                               text
0      2  copycat muslim terrorist arrested with assault...
1      2  wow! chicago protester caught on camera admits...
2      2   germany's fdp look to fill schaeuble's big shoes
3      2  mi school sends welcome back packet warning ki...
4      2  u.n. seeks 'massive' aid boost amid rohingya '...

Información del testing_data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9984 entries, 0 to 9983
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   marker  9984 non-null   object
 1   text    9984 non-null   object
dtypes: object(2)
memory usage: 156.1+ KB
None

Texto preprocesado del testing_data (ejemplo):
                                                text  \
0  copycat muslim terrorist arrested with assault...   
1  wow! chicago protester caught on camera admits...   
2   germany's fdp look to fill schaeuble's big shoes   
3  mi s

In [94]:
X_test_new_tfidf = tfidf_vectorizer.transform(df_test['cleaned_text'])
# Utiliza el vectorizador TF-IDF *entrenado* previamente con los datos de entrenamiento para transformar el texto preprocesado del conjunto de datos de prueba en la misma representación numérica.
# ¡Es crucial usar el mismo vectorizador para asegurar la consistencia de las características!
print("\nForma de la matriz TF-IDF para el testing_data:", X_test_new_tfidf.shape) # Muestra la forma de la matriz TF-IDF para el conjunto de datos de prueba.


Forma de la matriz TF-IDF para el testing_data: (9984, 5000)


In [95]:
y_pred_new_lr = logistic_regression_model.predict(X_test_new_tfidf)
# Utiliza el modelo de Regresión Logística *entrenado* previamente para hacer predicciones sobre los datos de prueba vectorizados.
print("\nPredicciones del modelo de Regresión Logística para testing_data:")
print(y_pred_new_lr) # Muestra las predicciones del modelo para el conjunto de datos de prueba.


Predicciones del modelo de Regresión Logística para testing_data:
[0 0 1 ... 1 0 0]


In [96]:
df_test['sentiment_score'] = df_test['text'].apply(lambda text: sentiment_analyzer.polarity_scores(text)['compound'])
# Aplica el análisis de sentimiento al conjunto de datos de prueba de la misma manera que se hizo con el conjunto de entrenamiento.
print("\nPuntajes de sentimiento para testing_data (ejemplo):")
print(df_test[['text', 'sentiment_score']].head()) # Muestra un ejemplo de los puntajes de sentimiento para el conjunto de datos de prueba.


Puntajes de sentimiento para testing_data (ejemplo):
                                                text  sentiment_score
0  copycat muslim terrorist arrested with assault...          -0.9382
1  wow! chicago protester caught on camera admits...           0.3382
2   germany's fdp look to fill schaeuble's big shoes           0.0000
3  mi school sends welcome back packet warning ki...           0.1531
4  u.n. seeks 'massive' aid boost amid rohingya '...          -0.3612


### Guardar el modelo

In [97]:
# Guardar el modelo entrenado
model_filename = 'fake_news_model.joblib' # Define el nombre del archivo donde se guardará el modelo.
joblib.dump(logistic_regression_model, model_filename) # Guarda el objeto del modelo de Regresión Logística en el archivo especificado.
print(f"Modelo guardado en: {model_filename}") # Informa al usuario de que el modelo se ha guardado.

# Guardar el vectorizador entrenado
vectorizer_filename = 'tfidf_vectorizer.joblib' # Define el nombre del archivo donde se guardará el vectorizador.
joblib.dump(tfidf_vectorizer, vectorizer_filename) # Guarda el objeto del vectorizador TF-IDF en el archivo especificado.
print(f"Vectorizador guardado en: {vectorizer_filename}") # Informa al usuario de que el vectorizador se ha guardado.

Modelo guardado en: fake_news_model.joblib
Vectorizador guardado en: tfidf_vectorizer.joblib
